In [1]:
from sortmaster.sort import Sort

c:\Users\risha\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
import cv2
from ultralytics import YOLOv10
import numpy as np
from sortmaster.sort import Sort

# Load YOLOv10 model
model = YOLOv10("best.pt")  # Replace with your YOLOv10 trained weights

# Initialize SORT tracker
tracker = Sort(max_age=40, min_hits=4, iou_threshold=0.4)

# Initialize video
video_path = "33198-396487799_tiny.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define video writer to save output
output_path = "output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Counting setup
counted_ids = set()
total_count = 0

# Create a named window with the proper size for cv2.imshow
cv2.namedWindow("Vehicle Counting", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Vehicle Counting", frame_width, frame_height)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform YOLOv10 inference
    results = model(frame)
    detections = []

    # Extract bounding boxes, confidences, and class IDs
    for result in results[0].boxes:
        box = result.xyxy[0].cpu().numpy()
        confidence = result.conf[0].cpu().numpy()
        class_id = int(result.cls[0].cpu().numpy())
        x1, y1, x2, y2 = map(int, box)
        width, height = x2 - x1, y2 - y1

        # Apply filtering
        if confidence > 0.5 and class_id == 2 and 50 <= width <= 500 and 50 <= height <= 500:
            detections.append([x1, y1, x2, y2, confidence])

    # Update SORT tracker
    tracked_objects = tracker.update(np.array(detections)) if len(detections) > 0 else []

    # Count new IDs
    for obj in tracked_objects:
        x1, y1, x2, y2, obj_id = map(int, obj)
        cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)

        # Check if the object ID is new
        if obj_id not in counted_ids:
            counted_ids.add(obj_id)  # Add the new ID to the counted set
            total_count += 1         # Increment the count

        # Draw bounding boxes and object ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {obj_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Draw total count
    cv2.putText(frame, f"Total Count: {total_count}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Write processed frame to output
    out.write(frame)

    # Show frame
    cv2.imshow("Vehicle Counting", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


c:\Users\risha\anaconda3\Lib\site-packages\ultralytics\nn\tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")



0: 384x640 2 Cars, 308.8ms
Speed: 8.2ms preprocess, 308.8ms inference, 12.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Cars, 257.6ms
Speed: 3.2ms preprocess, 257.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Cars, 245.1ms
Speed: 2.0ms preprocess, 245.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Cars, 254.2ms
Speed: 2.0ms preprocess, 254.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Cars, 253.0ms
Speed: 2.0ms preprocess, 253.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Cars, 263.3ms
Speed: 2.2ms preprocess, 263.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Cars, 260.6ms
Speed: 1.8ms preprocess, 260.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 Cars, 281.1ms
Speed: 1.1ms preprocess, 281.1ms inference, 0.7ms postprocess per image at shape (1, 3, 3

In [3]:
import cv2
from ultralytics import YOLOv10
import numpy as np
from sortmaster.sort import Sort

# ------------------------------
# Vehicle Counting and Average Calculation
# ------------------------------

# Load YOLOv10 model
model = YOLOv10("best.pt")  # Replace with your trained weights

# Initialize SORT tracker
tracker = Sort(max_age=40, min_hits=4, iou_threshold=0.4)

# Open video file
video_path = "33198-396487799_tiny.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
video_length = total_frames / fps  # in seconds

# Set up video writer to save output video
output_path = "output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Initialize counting variables
counted_ids = set()
total_count = 0

# Create a named window for display
cv2.namedWindow("Vehicle Counting", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Vehicle Counting", frame_width, frame_height)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv10 inference on the frame
    results = model(frame)
    detections = []

    # Extract detections and filter by confidence, class (e.g. class_id==2 for car), and bounding box size
    for result in results[0].boxes:
        box = result.xyxy[0].cpu().numpy()
        confidence = result.conf[0].cpu().numpy()
        class_id = int(result.cls[0].cpu().numpy())
        x1, y1, x2, y2 = map(int, box)
        width, height = x2 - x1, y2 - y1

        # Adjust filtering criteria as needed
        if confidence > 0.5 and class_id == 2 and 50 <= width <= 500 and 50 <= height <= 500:
            detections.append([x1, y1, x2, y2, confidence])

    # Update tracker with current detections
    tracked_objects = tracker.update(np.array(detections)) if len(detections) > 0 else []

    # Loop over tracked objects to count and draw bounding boxes
    for obj in tracked_objects:
        x1, y1, x2, y2, obj_id = map(int, obj)
        if obj_id not in counted_ids:
            counted_ids.add(obj_id)
            total_count += 1

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {obj_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the total count on the frame
    cv2.putText(frame, f"Total Count: {total_count}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    out.write(frame)
    cv2.imshow("Vehicle Counting", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

# Calculate average vehicles per second and convert to vehicles per hour
if video_length > 0:
    avg_vehicles_per_sec = total_count / video_length
else:
    avg_vehicles_per_sec = 0
vehicles_per_hour = int(avg_vehicles_per_sec * 3600)

print(f"Total vehicles: {total_count}")
print(f"Video length (seconds): {video_length:.2f}")
print(f"Average vehicles per second: {avg_vehicles_per_sec:.2f}")
print(f"Vehicles per hour (computed): {vehicles_per_hour}")

# ------------------------------
# Create and Save XML Route File
# ------------------------------

# Here we update the north-south flows with the computed vehicles_per_hour value.
# The east-west flows remain as in your template (300 vehsPerHour).
xml_content = f"""<?xml version="1.0" encoding="UTF-8"?>
<routes>
    <route id="route_ns" edges="n_t t_s"/>
    <route id="route_we" edges="w_t t_e"/>
    <route id="route_ew" edges="e_t t_w"/>
    <route id="route_sn" edges="s_t t_n"/>
    <flow id="flow_ns_0" route="route_ns" begin="0" end="100000" vehsPerHour="{vehicles_per_hour}" departSp100000d="max" departPos="base" departLane="best"/>
    <flow id="flow_sn_0" route="route_sn" begin="0" end="100000" vehsPerHour="{vehicles_per_hour}" departSp100000d="max" departPos="base" departLane="best"/>
    <flow id="flow_ew_0" route="route_ew" begin="0" end="100000" vehsPerHour="{vehicles_per_hour}" departSp100000d="max" departPos="base" departLane="best"/>
    <flow id="flow_we_0" route="route_we" begin="0" end="100000" vehsPerHour="{vehicles_per_hour}" departSp100000d="max" departPos="base" departLane="best"/>
</routes>
"""

# Save the XML content to a file
xml_file_path = "routes.xml"
with open(xml_file_path, "w") as f:
    f.write(xml_content)

print(f"Route file '{xml_file_path}' has been saved.")


c:\Users\risha\anaconda3\Lib\site-packages\ultralytics\nn\tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")



0: 384x640 2 Cars, 270.8ms
Speed: 9.1ms preprocess, 270.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Cars, 241.7ms
Speed: 2.7ms preprocess, 241.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Cars, 240.9ms
Speed: 1.9ms preprocess, 240.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Cars, 228.9ms
Speed: 2.0ms preprocess, 228.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Cars, 251.2ms
Speed: 1.1ms preprocess, 251.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Cars, 255.1ms
Speed: 1.7ms preprocess, 255.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Cars, 272.4ms
Speed: 1.7ms preprocess, 272.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 Cars, 271.4ms
Speed: 1.0ms preprocess, 271.4ms inference, 0.0ms postprocess per image at shape (1, 3, 38